#### This notebook evaluates the identity model on the Norman dataset.

The identity model (= "no effect model") assumes that perturbations have no effect and predictions thus correspond to control cells. These 'predictions' are computed on-the-fly in this notebook (rather than being loaded as in the other baseline models).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import jax
import os
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import anndata as ad
import pandas as pd
from tqdm.auto import tqdm

In [3]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [4]:
split = 3

#### Data loading

In [5]:
DATA_DIR = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019/norman_preprocessed_adata"

adata_train_path = os.path.join(DATA_DIR, f"adata_train_pca_3_split_{split}.h5ad")
adata_test_path = os.path.join(DATA_DIR, f"adata_val_pca_3_split_{split}.h5ad")
adata_ood_path = os.path.join(DATA_DIR, f"adata_test_pca_3_split_{split}.h5ad")

# load data splits
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

#### Add subgroup annotations

- single: Only one gene has been perturbed. This perturbation is not part of the training data.
- double_seen_0: Two genes have been perturbed. Neither of these gene perturbations have been seen as single perturbations in the training data.
- double_seen_1: Two genes have been perturbed. One of the two perturbed genes has been seen as single perturbation in the training data.
- double_seen_2: Two genes have been perturbed. Both gene perturbations have been seen as single perturbation in the training data.

In [6]:
train_conditions = adata_train.obs.condition.str.replace("+ctrl", "").str.replace("ctrl+", "").unique()

assert not adata_ood[adata_ood.obs.condition != "ctrl"].obs.condition.isin(train_conditions).any()

mask_single_perturbation = adata_ood.obs.condition.str.contains("ctrl")
mask_double_perturbation_seen_0 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    ~adata_ood.obs.gene_1.isin(train_conditions) & 
    ~adata_ood.obs.gene_2.isin(train_conditions)
)
mask_double_perturbation_seen_1 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    (
        (adata_ood.obs.gene_1.isin(train_conditions) & ~adata_ood.obs.gene_2.isin(train_conditions)) | 
        (~adata_ood.obs.gene_1.isin(train_conditions) & adata_ood.obs.gene_2.isin(train_conditions))
    )
)
mask_double_perturbation_seen_2 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    adata_ood.obs.gene_1.isin(train_conditions) & 
    adata_ood.obs.gene_2.isin(train_conditions)
)
adata_ood.obs.loc[mask_single_perturbation, "subgroup"] = "single"
adata_ood.obs.loc[mask_double_perturbation_seen_0, "subgroup"] = "double_seen_0"
adata_ood.obs.loc[mask_double_perturbation_seen_1, "subgroup"] = "double_seen_1"
adata_ood.obs.loc[mask_double_perturbation_seen_2, "subgroup"] = "double_seen_2"

display(adata_ood.obs.subgroup.value_counts())

subgroup
double_seen_1    13587
single           11685
double_seen_2     2815
double_seen_0     1873
Name: count, dtype: int64

In [7]:
# compute pca on full dataset
adata_all = ad.concat((adata_train, adata_test, adata_ood))
cfpp.centered_pca(adata_all, n_comps=10)

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


#### Predict on test set

In [8]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_test = adata_test[adata_test.obs["condition"]=="ctrl"]

# project predictions and ground truth data onto pca space
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_all)

# dict for ground truths per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded, test_data_target_decoded = {}, {}

# dict for predictions per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded_predicted, test_data_target_decoded_predicted = {}, {}

for cond in adata_test.obs["condition"].cat.categories:
    if cond == "ctrl":
        continue

    # pca space
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_encoded_predicted[cond] = adata_pred_test.obsm["X_pca"]
    
    # gene space
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X
    test_data_target_decoded_predicted[cond] = adata_pred_test.X

/ictstr01/home/haicu/soeren.becker/repos/cell_flow_perturbation/src/cfp/preprocessing/_pca.py:192: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  query_adata.obsm[obsm_key_added] = np.array(


#### Predict on ood set (full ood set + subgroups)

In [9]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_ood = adata_ood[adata_ood.obs["condition"]=="ctrl"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_all)

ood_data_target_encoded, ood_data_target_decoded = {}, {}
ood_data_target_encoded_predicted, ood_data_target_decoded_predicted = {}, {}

subgroups = ["all", "single", "double_seen_0", "double_seen_1", "double_seen_2"]

for subgroup in subgroups:

    ood_data_target_encoded[subgroup] = {}
    ood_data_target_decoded[subgroup] = {}
    ood_data_target_encoded_predicted[subgroup] = {}
    ood_data_target_decoded_predicted[subgroup] = {}
    
    for cond in adata_ood.obs["condition"].cat.categories:
        if cond == "ctrl":
            continue
        
        select = adata_ood.obs["condition"] == cond
        if subgroup != "all":
            select = select & (adata_ood.obs.subgroup == subgroup)

        if not any(select):
            # the condition is not part of this subgroup
            continue
        
        # pca space
        ood_data_target_encoded[subgroup][cond] = adata_ood[select].obsm["X_pca"]
        ood_data_target_encoded_predicted[subgroup][cond] = adata_pred_ood.obsm["X_pca"]
    
        # gene space
        ood_data_target_decoded[subgroup][cond] = adata_ood[select].X
        ood_data_target_decoded_predicted[subgroup][cond] = adata_pred_ood.X

#### Evaluation on test set

In [10]:
# test set: evaluation in encoded (=pca) space
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    test_data_target_encoded, 
    test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(
    test_metrics_encoded, 
    prefix="encoded_test_"
)

# test set: evaluation in decoded (=gene) space
test_metrics_decoded = jax.tree_util.tree_map(
    # compute_metrics, 
    compute_metrics_fast, 
    test_data_target_decoded, 
    test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(
    test_metrics_decoded, 
    prefix="decoded_test_"
)

# test set
test_deg_dict = {
    k: v 
    for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
    if k in test_data_target_decoded_predicted.keys()
}

# test set
test_deg_target_decoded_predicted = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded_predicted, 
    test_deg_dict
)
test_deg_target_decoded = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded, 
    test_deg_dict
)

deg_test_metrics = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    test_deg_target_decoded, 
    test_deg_target_decoded_predicted
)
deg_mean_test_metrics = compute_mean_metrics(
    deg_test_metrics, 
    prefix="deg_test_"
)

E1114 16:02:40.052095  819073 hlo_lexer.cc:443] Failed to parse int literal: 158860238866516663516


In [11]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -1.0193046170931597,
 'encoded_test_sinkhorn_div_1': 20.300258893233078,
 'encoded_test_sinkhorn_div_10': 16.064701006962704,
 'encoded_test_sinkhorn_div_100': 13.464954082782452,
 'encoded_test_e_distance': 25.898587002317086,
 'encoded_test_mmd': 0.061631125469620414}

In [12]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.955407589673996,
 'decoded_test_sinkhorn_div_1': 118.30650446965144,
 'decoded_test_sinkhorn_div_10': 84.69106190021222,
 'decoded_test_sinkhorn_div_100': 17.86936803964468,
 'decoded_test_e_distance': 32.66198071051835,
 'decoded_test_mmd': 0.027154897894853584}

In [13]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.7863385127140925,
 'deg_test_sinkhorn_div_1': 23.670191718981815,
 'deg_test_sinkhorn_div_10': 13.803049701910753,
 'deg_test_sinkhorn_div_100': 11.990354427924522,
 'deg_test_e_distance': 23.58044464832553,
 'deg_test_mmd': 0.06124206571481549}

#### Evaluation on ood set

In [14]:
ood_metrics_encoded, mean_ood_metrics_encoded = {}, {}
ood_metrics_decoded, mean_ood_metrics_decoded = {}, {}
deg_ood_metrics, deg_mean_ood_metrics = {}, {}
ood_deg_dict = {}
ood_deg_target_decoded_predicted, ood_deg_target_decoded = {}, {}

for subgroup in tqdm(subgroups[::-1]):

    print(f"subgroup: {subgroup}")

    print("Computing ood_metrics_encoded")
    # ood set: evaluation in encoded (=pca) space
    ood_metrics_encoded[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_data_target_encoded[subgroup], 
        ood_data_target_encoded_predicted[subgroup]
    )
    mean_ood_metrics_encoded[subgroup] = compute_mean_metrics(
        ood_metrics_encoded[subgroup], 
        prefix="encoded_ood_",
    )

    print("Computing ood_metrics_decoded")
    # ood set: evaluation in decoded (=gene) space
    ood_metrics_decoded[subgroup] = jax.tree_util.tree_map(
        # compute_metrics, 
        compute_metrics_fast, 
        ood_data_target_decoded[subgroup], 
        ood_data_target_decoded_predicted[subgroup]
    )
    mean_ood_metrics_decoded[subgroup] = compute_mean_metrics(
        ood_metrics_decoded[subgroup], 
        prefix="decoded_ood_",
    )

    # ood set
    ood_deg_dict[subgroup] = {
        k: v
        for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
        if k in ood_data_target_decoded_predicted[subgroup].keys()
    }

    print("Apply DEG mask")
    # ood set
    ood_deg_target_decoded_predicted[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded_predicted[subgroup], 
        ood_deg_dict[subgroup]
    )
    
    ood_deg_target_decoded[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded[subgroup], 
        ood_deg_dict[subgroup]
    )

    print("Compute metrics on DEG subsetted decoded")
    deg_ood_metrics[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_deg_target_decoded[subgroup], 
        ood_deg_target_decoded_predicted[subgroup]
    )
    deg_mean_ood_metrics[subgroup] = compute_mean_metrics(
        deg_ood_metrics[subgroup], 
        prefix="deg_ood_"
    )

  0%|          | 0/5 [00:00<?, ?it/s]

subgroup: double_seen_2
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_1
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_0
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: single
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: all
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded


In [15]:
mean_ood_metrics_encoded

{'double_seen_2': {'encoded_ood_r_squared': -0.8182484656572342,
  'encoded_ood_sinkhorn_div_1': 28.310436248779297,
  'encoded_ood_sinkhorn_div_10': 23.292140165964764,
  'encoded_ood_sinkhorn_div_100': 19.369065284729004,
  'encoded_ood_e_distance': 37.40626049025098,
  'encoded_ood_mmd': 0.08756249817088246},
 'double_seen_1': {'encoded_ood_r_squared': -1.2104228375092991,
  'encoded_ood_sinkhorn_div_1': 26.64607388118528,
  'encoded_ood_sinkhorn_div_10': 21.941482184068214,
  'encoded_ood_sinkhorn_div_100': 18.14314741458533,
  'encoded_ood_e_distance': 34.57573491095868,
  'encoded_ood_mmd': 0.07594861771981672},
 'double_seen_0': {'encoded_ood_r_squared': -1.151916742324829,
  'encoded_ood_sinkhorn_div_1': 22.576999346415203,
  'encoded_ood_sinkhorn_div_10': 18.06695704989963,
  'encoded_ood_sinkhorn_div_100': 15.64321157667372,
  'encoded_ood_e_distance': 30.625814467552896,
  'encoded_ood_mmd': 0.07749342173337936},
 'single': {'encoded_ood_r_squared': -0.7675928151285326,
  'e

In [16]:
mean_ood_metrics_decoded

{'double_seen_2': {'decoded_ood_r_squared': 0.9355401496092478,
  'decoded_ood_sinkhorn_div_1': 135.986302057902,
  'decoded_ood_sinkhorn_div_10': 103.42467721303304,
  'decoded_ood_sinkhorn_div_100': 25.884009679158527,
  'decoded_ood_e_distance': 46.74367822386795,
  'decoded_ood_mmd': 0.03780840508018931},
 'double_seen_1': {'decoded_ood_r_squared': 0.9386914822290529,
  'decoded_ood_sinkhorn_div_1': 132.44223267177367,
  'decoded_ood_sinkhorn_div_10': 99.45550738640551,
  'decoded_ood_sinkhorn_div_100': 25.094286072928952,
  'decoded_ood_e_distance': 44.56089625470371,
  'decoded_ood_mmd': 0.034479049825162256},
 'double_seen_0': {'decoded_ood_r_squared': 0.948093851407369,
  'decoded_ood_sinkhorn_div_1': 121.41927337646484,
  'decoded_ood_sinkhorn_div_10': 88.63462914360895,
  'decoded_ood_sinkhorn_div_100': 20.58209228515625,
  'decoded_ood_e_distance': 38.44946936663116,
  'decoded_ood_mmd': 0.034688023229440056},
 'single': {'decoded_ood_r_squared': 0.9743763288936099,
  'decod

In [17]:
deg_mean_ood_metrics

{'double_seen_2': {'deg_ood_r_squared': 0.7668259143829346,
  'deg_ood_sinkhorn_div_1': 30.4760320186615,
  'deg_ood_sinkhorn_div_10': 18.585283915201824,
  'deg_ood_sinkhorn_div_100': 16.298033873240154,
  'deg_ood_e_distance': 32.166943264189285,
  'deg_ood_mmd': 0.08063141939540704},
 'double_seen_1': {'deg_ood_r_squared': 0.737968920536761,
  'deg_ood_sinkhorn_div_1': 26.19584974140491,
  'deg_ood_sinkhorn_div_10': 15.810027273196095,
  'deg_ood_sinkhorn_div_100': 13.948523683368036,
  'deg_ood_e_distance': 27.446497184981645,
  'deg_ood_mmd': 0.07810710504088762},
 'double_seen_0': {'deg_ood_r_squared': 0.8150879608260261,
  'deg_ood_sinkhorn_div_1': 24.048120127783882,
  'deg_ood_sinkhorn_div_10': 14.664608107672798,
  'deg_ood_sinkhorn_div_100': 13.530794779459635,
  'deg_ood_e_distance': 26.883714785596194,
  'deg_ood_mmd': 0.0861144144501951},
 'single': {'deg_ood_r_squared': 0.8664752679902155,
  'deg_ood_sinkhorn_div_1': 16.217136230017687,
  'deg_ood_sinkhorn_div_10': 7.593

In [18]:
collected_results = {
    # test
    "test_metrics_encoded": test_metrics_encoded,
    "mean_test_metrics_encoded": mean_test_metrics_encoded,
    "test_metrics_decoded": test_metrics_decoded, 
    "mean_test_metrics_decoded": mean_test_metrics_decoded,
    "test_deg_dict": test_deg_dict,
    "test_deg_target_decoded_predicted": test_deg_target_decoded_predicted,
    "test_deg_target_decoded": test_deg_target_decoded,
    "deg_test_metrics": deg_test_metrics,
    "deg_mean_test_metrics": deg_mean_test_metrics,
    # ood
    "ood_metrics_encoded": ood_metrics_encoded,
    "mean_ood_metrics_encoded": mean_ood_metrics_encoded,
    "ood_metrics_decoded": ood_metrics_decoded,
    "mean_ood_metrics_decoded": mean_ood_metrics_decoded,
    "deg_ood_metrics": deg_ood_metrics,
    "deg_mean_ood_metrics": deg_mean_ood_metrics,
    "ood_deg_dict": ood_deg_dict,
    "ood_deg_target_decoded_predicted": ood_deg_target_decoded_predicted,
    "ood_deg_target_decoded": ood_deg_target_decoded,
}

In [19]:
OUT_DIR = "/lustre/groups/ml01/workspace/ot_perturbation/data/norman_soren/identity"
pd.to_pickle(collected_results, os.path.join(f"norman_identity_split_{split}_collected_results.pkl"))